# TIATNIC
---
Competição do Kaggle sobre dados do Titanic disponível em:</br>
https://www.kaggle.com/competitions/titanic/data</br>
</br>
São dispostas diversas informações individualizadas sobre cada passageiro do navio</br>
com o intiuito de predizer quem sobreviveu ou não à trajédia que envolveu o navio.</br>
</br>
São oferecidos dois conjuntos de dados, um de treino que além das informações de cada</br>
passageiro, contém como resposta a informação dos sobreviventes.</br>
O outro conjunto é tomado como teste e contém as mesmas informações dos passageiros que</br>
o anterior mas não contém a classificação dos sobreviventes.</br>
</br>
O objetivo é estabelecer, utilizando a base de treino como auxílio, a resposta para a base</br>
de teste e enviar um csv contendo apenas os ids e a resposta sobre a sobrevivência como um dado binário.

---
## 1) IMPORTAÇÃO DAS BIBLIOTECAS UTILIZADAS
---

Antes de iniciar, os Warnings serão omitidos contibuindo na visualização.

In [90]:
import warnings
warnings.filterwarnings('ignore')

Bloco contendo todas as importações para falitar o acompanhamento e padronização.

In [91]:
import pandas as pd                                         # Manuseio dos DataFrames
from ydata_profiling import ProfileReport                   # Relatório automático para compreensão dos dados
from sklearn.model_selection import train_test_split        # Separação de uma base em treino e teste
from sklearn import tree                                    # Árvore de decisão
from sklearn.neighbors import KNeighborsClassifier          # KNN
from sklearn.linear_model import LogisticRegression         # Regressão logística
from sklearn.metrics import accuracy_score                  # Acurácia
from sklearn.metrics import confusion_matrix                # Matriz de confusão
import matplotlib.pyplot as plt                             # Biblioteca gráffica
from sklearn.preprocessing import RobustScaler              # Ajustar escala para o BoxPlot
import seaborn as sns                                       # Biblioteca gráffica
from sklearn.ensemble import RandomForestClassifier         # Random Forest
from sklearn.neural_network import MLPClassifier            # MLP
from datetime import datetime                               # Biblioteca para contar o tempo
from sklearn.model_selection import KFold                   # KFold
from sklearn.model_selection import GridSearchCV            # Grid Search

## 2) ENTENDENDO OS DADOS
---
Antes de qualquer tratamento, é necessário compreender melhor os dados que estão</br>
sendo oferecidos e aí então a trajetória ficará mais clara.
### Leitura dos dados

In [92]:
test = pd.read_csv('DATA/test.csv', sep=',',  on_bad_lines='skip')
train = pd.read_csv('DATA/train.csv', sep=',',  on_bad_lines='skip')
train.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


### Produção de um relatório em Html para facilitar a compreensão da base de dados
O ProfileReport cria, facilmente, um relatório dos dados e o exporta em html.</br>
Isso é muito importante para a compreensão do conjunto e visualização dos dados.

In [93]:
profile = ProfileReport(train, title='Relatório de Perfil - Titanic')
profile.to_file("relatorio_titanic.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 12/12 [00:00<00:00, 382.76it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Observações gerais e complementares
Após a observação do relatório e uma primeira visão mais a fundo dos dados</br>
algumas verificações são necessárias, como:
* tipo dos dados;
* Dados nulos;
* Comparação entre as bases;

In [94]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [95]:
train.dtypes.value_counts()

int64      5
object     5
float64    2
Name: count, dtype: int64

In [96]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [97]:
test.dtypes.value_counts()

object     5
int64      4
float64    2
Name: count, dtype: int64

As bases de treino e teste são iguais no tipo dos dados<br>
As variáveis Name, Sex, Ticket, Cabin e Embarked são do tipo object.<br>
Então estas variáveis precisarão ser observadas e tratadas</br>
</br>
Já é possível notar que existem valores nulos.
### Valores nulos

In [98]:
train.isnull().sum().sort_values(ascending=False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Name             0
Pclass           0
Survived         0
Sex              0
Parch            0
SibSp            0
Fare             0
Ticket           0
dtype: int64

In [99]:
test.isnull().sum().sort_values(ascending=False)

Cabin          327
Age             86
Fare             1
Name             0
Pclass           0
PassengerId      0
Sex              0
Parch            0
SibSp            0
Ticket           0
Embarked         0
dtype: int64

Tanto na base de treino como na base de teste <br>
existe um grande número de valores nulos na variável Cabin.<br>
Logo, este fato implicará em sua exclusão.</br>
</br>
Tanto Age quanto Fare, em ambas as bases terão os valores nulos substituídos pela sua média.</br>
<br>
Embarked na base de treino terá substituído os valores nulos pela moda.
## 3) TRATAMENTOS INICIAIS
---
### Cardinalidade dos dados
Dados com alta cardinalidade não coreespondem bem à buscas por padões.

In [100]:
train.nunique().sort_values(ascending=False)

PassengerId    891
Name           891
Ticket         681
Fare           248
Cabin          147
Age             88
SibSp            7
Parch            7
Embarked         3
Pclass           3
Survived         2
Sex              2
dtype: int64

In [101]:
test.nunique().sort_values(ascending=False)

PassengerId    418
Name           418
Ticket         363
Fare           169
Age             79
Cabin           76
Parch            8
SibSp            7
Pclass           3
Embarked         3
Sex              2
dtype: int64

O PassengerId não será considerado nas predições.</br>
Apenas será mantido na base de teste para o envio para o Kaggle.</br>
</br>
A variável Name possui um valor diferente para cada passageiro.</br>
Como nome é realmente único ou sem conexão quando igual, esta poderá ser excluída.</br>
</br>
Ticket possui alta cardinalidade e nenhum agrupamento possível, logo será descartado.</br>
</br>
Fare e Age poderão ser repensados para representar melhor o modelo.</br>
</br>
Cabin já possui alto número de nulos, junto com a alta cardinalidade será desconsiderada.

In [102]:
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)
test = test.drop(['Name', 'Ticket', 'Cabin'], axis = 1)

In [103]:
train.head(1)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.25,S


### Dados nulos
Tratamento dos dados nulos já observados.<br>
<br>
**Idade**<br>
Os valores nuos da idade seráo substituídos pela média do valor.<br>
Ambas as bases possuem valores nulos que serão tratados da mesma maneira.

In [104]:
train.Age.mean()

np.float64(29.69911764705882)

In [105]:
train.loc[train.Age.isnull(), 'Age'] = train.Age.mean()
test.loc[test.Age.isnull(), 'Age'] = test.Age.mean()

**Embarked** <br>
Os valores nulos em embarked serão substituídos pela moda.<br>
Apenas a base de treino possuí nulo nesta variável.

In [106]:
train.Embarked.mode()

0    S
Name: Embarked, dtype: object

In [107]:
train.loc[train.Embarked.isnull(), 'Embarked'] = train.Embarked.mode()[0]

**Fare**<br>
Os valores nulo em Fare seráo substituídos pela média.<br>
Somente a base de teste possuí nulos nesta variável.

In [108]:
test.Fare.mean()

np.float64(35.627188489208635)

In [109]:
test.loc[test.Fare.isnull(), 'Fare'] = test.Fare.mean()

#### Conferindo se os dados foram tratados

In [110]:
train.isnull().sum().sort_values(ascending=False)

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [111]:
test.isnull().sum().sort_values(ascending=False)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

## 4) Aplicação de modelos nos dados numéricos
---
Um subconjunto será criado contendo apenas os dados numéricos.<br>
Assim, Árvore de decisão, Knn e Regressão Logística poderão ser aplicadas neste conjunto.

In [112]:
train_nr = train.loc[:, train.columns[train.dtypes != 'object']]
test_nr = test.loc[:, test.columns[test.dtypes != 'object']]

Criação do conjunto X e y, sendo X os dados de treino sem o Survived e y, como resposta, o Survived.<br>
Em seguida, separação dos grupos de treino e validação para a base de treino.

In [113]:
X = train_nr.drop(['Survived'], axis = 1)
y = train.Survived

In [114]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

### Árevore de classificação

In [115]:
clf_tree = tree.DecisionTreeClassifier(random_state = 42)
clf_tree = clf_tree.fit(X_train, y_train)
pred_tree = clf_tree.predict(X_val)

### KNN

In [116]:
clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_knn = clf_knn.fit(X_train, y_train)
pred_knn = clf_knn.predict(X_val)

### Regressão logística

In [117]:
clf_rl = LogisticRegression(random_state=42)
clf_rl = clf_rl.fit(X_train, y_train)
pred_log = clf_rl.predict(X_val)

### Acurácia

In [118]:
acuracia_tree = accuracy_score(y_val, pred_tree)
acuracia_knn = accuracy_score(y_val, pred_knn)
acuracia_rl = accuracy_score(y_val, pred_log)

acuracia_dict = {
    'Modelos' : ['Árvore de decisão', 'KNN', 'Regressão logística'],
    'Somente Numéricos' : [acuracia_tree, acuracia_knn, acuracia_rl]
}
acuracia = pd.DataFrame.from_dict(acuracia_dict)

acuracia

,Modelos,Somente Numéricos
0,Árvore de decisão,0.616949
1,KNN,0.667797
2,Regressão logística,0.725424


### Matriz de confusão
* Árvore

In [119]:
confusion_matrix(y_val, pred_tree)

array([[125,  50],
       [ 63,  57]])

* Knn

In [120]:
confusion_matrix(y_val, pred_knn)

array([[134,  41],
       [ 57,  63]])

* Regressão logística

In [121]:
confusion_matrix(y_val, pred_log)

array([[156,  19],
       [ 62,  58]])

### Aplicação nos dados de teste propostos pelo desafio
Os maiores resultados foram obtidos pela Regressão Logística.<br>
Este modelo será aplicado aos dados numéricos da base de teste e enviados ao Kaggle.

In [122]:
df_desafio = pd.DataFrame()
df_desafio['PassengerId'] = test_nr['PassengerId']
X_teste = test_nr.drop(['PassengerId'], axis = 1)
y_pred = clf_rl.predict(X_teste)
df_desafio['Survived'] = y_pred

df_desafio.to_csv('Export/only_numbers_knn.csv', index = False)

<img src="IMG/im1.png" width="500" height="800">
O resultado obtido nesta submissão foi 0.66</br>

## 5) NOVO TRATAMENTO DOS DADOS
---
Colunas que não são numéricas são tratadas e incluídas neste passo.

In [123]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [124]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    object 
 3   Age          418 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Fare         418 non-null    float64
 7   Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 26.3+ KB


### Coluna SEX
Esta coluna é binária, então apenas será passada para números 0 e 1.

In [125]:
train.Sex.value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

*    male= 1 <br>
*    female=0

In [126]:
train['Sex'] = train.Sex.apply(lambda x: 1 if x == 'male' else 0)
test['Sex'] = test.Sex.apply(lambda x: 1 if x == 'male' else 0)

Conferindo o resultado

In [127]:
train['Sex'].value_counts()

Sex
1    577
0    314
Name: count, dtype: int64

### Coluna Embarked
Para esta coluna será utilizado o One Hot Encoding.</br>
Embarked é composto por S, C e Q. </br>
Serão criadas três colunas, uma para cada valor, e um resultado binário será adicionado.

In [128]:
train['Embarked_S'] = train.Embarked.apply(lambda x: 1 if x == 'S' else 0)
train['Embarked_C'] = train.Embarked.apply(lambda x: 1 if x == 'C' else 0)
train['Embarked_Q'] = train.Embarked.apply(lambda x: 1 if x == 'Q' else 0)

test['Embarked_S'] = test.Embarked.apply(lambda x: 1 if x == 'S' else 0)
test['Embarked_C'] = test.Embarked.apply(lambda x: 1 if x == 'C' else 0)
test['Embarked_Q'] = test.Embarked.apply(lambda x: 1 if x == 'Q' else 0)

In [129]:
train[['Embarked', 'Embarked_S', 'Embarked_C', 'Embarked_Q']].value_counts()

Embarked  Embarked_S  Embarked_C  Embarked_Q
S         1           0           0             646
C         0           1           0             168
Q         0           0           1              77
Name: count, dtype: int64

In [130]:
train = train.drop(['Embarked'], axis = 1)
test = test.drop(['Embarked'], axis = 1)

## 6) Aplicação dos modelos ao novo tratamento de dados
---

In [131]:
X = train.drop(['Survived'], axis=1)
y = train.Survived

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

clf_tree = tree.DecisionTreeClassifier(random_state = 42)
clf_tree = clf_tree.fit(X_train, y_train)
pred_tree = clf_tree.predict(X_val)

clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_knn .fit(X_train, y_train)
pred_knn = clf_knn .predict(X_val)

clf_rl = LogisticRegression(max_iter = 500, random_state=42)
clf_rl.fit(X_train, y_train)
pred_log = clf_rl.predict(X_val)

### Acurácia

In [132]:
acuracia_tree = accuracy_score(y_val, pred_tree)
acuracia_knn = accuracy_score(y_val, pred_knn)
acuracia_rl = accuracy_score(y_val, pred_log)

acuracia['Sex e OHE'] = [acuracia_tree, acuracia_knn, acuracia_rl]

acuracia

,Modelos,Somente Numéricos,Sex e OHE
0,Árvore de decisão,0.616949,0.738983
1,KNN,0.667797,0.705085
2,Regressão logística,0.725424,0.816949


Notamos um aumento na acurácia de todos os métodos</br>
Principalmente em Reg. Logística com aumento de 9%
### Exportação Kaggle
Com a maior acurácia, o classificador Reg. Logística será escolhido para enviar os dados ao Kaggle.

In [133]:
df_desafio = pd.DataFrame()
df_desafio['PassengerId'] = test['PassengerId']
X_test = test.drop(['PassengerId'], axis = 1)
y_pred = clf_rl.predict(X_test)
df_desafio['Survived'] = y_pred

df_desafio.to_csv('Export/sexeohe.csv', index = False)

<img src="IMG/im2.png" width="500" height="800">
O resultado obtido na nova submissão foi 0.76 e aumentou 10%</br>

## 7) VISUALIZAÇÃO DOS DADOS OBTIDOS ATÉ O MOMENTO
---

In [134]:
fig, ax = plt.subplots(figsize=(10,5))

ax.boxplot(train.iloc[:,1:10])
ax.set_xticks(range(1,train.iloc[:,1:10].shape[1]+1),train.iloc[:,1:10].columns)

plt.show()

As colunas Age e Fare são muito diferentes das demais.</br>
Também, possuem grande número de Outliers.<br>
O mais indicado a se utilizar é ROBUSTSCALER.</br>
<br>
Existem diversos outros métodos que poderiam ser utilizados.

In [135]:
transformer = RobustScaler().fit(train[['Age', 'Fare']])
train[['Age', 'Fare']] = transformer.transform(train[['Age', 'Fare']])

transformer = RobustScaler().fit(test[['Age', 'Fare']])
test[['Age', 'Fare']] = transformer.transform(test[['Age', 'Fare']])

In [136]:
fig, ax = plt.subplots(figsize=(10,5))
ax.boxplot(train.iloc[:,1:10])
ax.set_xticks(range(1,train.iloc[:,1:10].shape[1]+1),train.iloc[:,1:10].columns)
plt.show()

Após a transformação a escala dos dados fica bem próxima

## 8) APLICAÇÃO DOS RESULTADOS COM A ESCALA AJUSTADA
---

In [137]:
X = train.drop(['Survived'], axis = 1)
y = train.Survived

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

clf_tree = tree.DecisionTreeClassifier(random_state = 42)
clf_tree = clf_tree.fit(X_train, y_train)
pred_tree = clf_tree.predict(X_val)

clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn .fit(X_train, y_train)
pred_knn = clf_knn .predict(X_val)

clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)
pred_log = clf.predict(X_val)

### Acurácia

In [138]:
acuracia_tree = accuracy_score(y_val, pred_tree)
acuracia_knn = accuracy_score(y_val, pred_knn)
acuracia_rl = accuracy_score(y_val, pred_log)

acuracia['Escala'] = [acuracia_tree, acuracia_knn, acuracia_rl]

acuracia

,Modelos,Somente Numéricos,Sex e OHE,Escala
0,Árvore de decisão,0.616949,0.738983,0.738983
1,KNN,0.667797,0.705085,0.800000
2,Regressão logística,0.725424,0.816949,0.816949


Houve aumento na acurácia apenas do KNN.</br>
Para esta técnica a acurácia nos dados de treino aumentou 10%.
### Exportação Kaggle
Apesar de apresentar uma acurácia um pouco menos, o KNN será escolhido </br>
para exportar os dados para o Kaggle já que teve grande aumento da acurácia<br>
assim o resultado poderá ser melhor observado.

In [139]:
df_desafio = pd.DataFrame()
df_desafio['PassengerId'] = test['PassengerId']
X_test = test.drop(['PassengerId'], axis = 1)
y_pred = clf_knn.predict(X_test)
df_desafio['Survived'] = y_pred

df_desafio.to_csv('Export/escala.csv', index = False)

<img src="IMG/im3.png" width="500" height="800">

Apesar de um aumento significativo na acurácia do KNN o resultado ainda</br>
está inferior ao alcançado pela Regressão Logística.

## 9) TRATAMENTO DE OUTRAS VARIÁVEIS
---
### Colunas SibSp e Parch

SibSp: Num de cônjuges ou irmãos à bordo</br>
Parch: Num de Pais ou Filhos à bordo</br>

### Verificar quantas pessoas sobreviveram nessas colunas

In [140]:
SibSp_df = train.groupby('SibSp')['Survived'].agg(['sum', 'count', 'mean']).reset_index()
SibSp_df.columns = ['SibSp', 'Sobrev', 'Total', 'Taxa']
SibSp_df

,SibSp,Sobrev,Total,Taxa
0,0,210,608,0.345395
1,1,112,209,0.535885
2,2,13,28,0.464286
3,3,4,16,0.250000
4,4,3,18,0.166667
5,5,0,5,0.000000
6,8,0,7,0.000000


In [141]:
Parch_df = train.groupby('Parch')['Survived'].agg(['sum', 'count', 'mean']).reset_index()
Parch_df.columns = ['Parch', 'Sobrev', 'Total', 'Taxa']
Parch_df

,Parch,Sobrev,Total,Taxa
0,0,233,678,0.343658
1,1,65,118,0.550847
2,2,40,80,0.500000
3,3,3,5,0.600000
4,4,0,4,0.000000
5,5,1,5,0.200000
6,6,0,1,0.000000


### Visualizando essas informações gráficamente

In [142]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10,6))

ax[0,0].plot(SibSp_df.SibSp , SibSp_df.Sobrev)
ax[0,0].plot(SibSp_df.SibSp , SibSp_df.Total)
ax[0,0].set_title('Sobreviventes por num de irmãos ou cônjuges')

labels1 = ax[0,1].bar(SibSp_df.SibSp , SibSp_df.Taxa)
ax[0,1].bar_label(labels1, fmt='%.02f')
ax[0,1].set(ylim=(0,0.6))
ax[0,1].set_title('Taxa de sobreviventes por num de irmãos ou cônjuges')

ax[1,0].plot(Parch_df.Parch , Parch_df.Sobrev)
ax[1,0].plot(Parch_df.Parch , Parch_df.Total)
ax[1,0].set_title('Sobreviventes por num de pais ou filhos')

labels2 = ax[1,1].bar(Parch_df.Parch , Parch_df.Taxa)
ax[1,1].bar_label(labels2, fmt='%.02f')
ax[1,1].set(ylim=(0,0.7))
ax[1,1].set_title('Tava de sobreviventes por num de pais ou filhos')

plt.show()

Essa análise permite notar que passageiros que viajaram só tem menos chance de sobrevivência.</br>
Comparando a taxa de sob para pessoas só.</br>
Com isso podemos inserir esta percepção em uma nova coluna no DataFrame.
### Inserção de nova variável

In [143]:
def sozinho(a,b):
    if a==0 and b==0 :
        return 1
    return 0

In [144]:
train['Sozinho'] = train.apply(lambda x: sozinho(x.SibSp, x.Parch), axis=1)
test['Sozinho'] = test.apply(lambda x: sozinho(x.SibSp, x.Parch), axis=1)

train.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_S,Embarked_C,Embarked_Q,Sozinho
0,0,3,1,-0.592240,1,0,-0.312011,1,0,0,0
1,1,1,0,0.638529,1,0,2.461242,0,1,0,0


### Variável Familiares
É Possível unificar as varáveis SibSp e Parch em uma única que se chamará familiar.</br>
Já que ambas descrevem os membros de uma mesma família que estão acompanhando o passageiro.

In [145]:
train['Familiares'] = train.SibSp + train.Parch
test['Familiares'] = test.SibSp + test.Parch

Assim serão removidas ambas as colunas que estão representadas por Familiares.

In [146]:
train = train.drop(['SibSp'], axis = 1)
train = train.drop(['Parch'], axis = 1)

test = test.drop(['SibSp'], axis = 1)
test = test.drop(['Parch'], axis = 1)

### Observando o comportamento dessa nova variável

In [147]:
Familiares_df = train.groupby('Familiares')['Survived'].agg(['sum', 'count', 'mean']).reset_index()
Familiares_df.columns = ['Familiares', 'Sobrev', 'Total', 'Taxa']
Familiares_df

,Familiares,Sobrev,Total,Taxa
0,0,163,537,0.303538
1,1,89,161,0.552795
2,2,59,102,0.578431
3,3,21,29,0.724138
4,4,3,15,0.200000
5,5,3,22,0.136364
6,6,4,12,0.333333
7,7,0,6,0.000000
8,10,0,7,0.000000


In [148]:
plt.bar(Familiares_df.Familiares , Familiares_df.Taxa)
plt.show

<function matplotlib.pyplot.show(*, block=None)>

## 10) COLETANDO RESULTADOS APÓS AS TRANSFORMAÇÕES
---

In [149]:
X = train.drop(['Survived'], axis = 1)
y = train.Survived

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

clf_tree = tree.DecisionTreeClassifier(random_state = 42)
clf_tree = clf_tree.fit(X_train, y_train)
pred_tree = clf_tree.predict(X_val)

clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn .fit(X_train, y_train)
pred_knn = clf_knn .predict(X_val)

clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)
pred_log = clf.predict(X_val)

In [150]:
acuracia_tree = accuracy_score(y_val, pred_tree)
acuracia_knn = accuracy_score(y_val, pred_knn)
acuracia_rl = accuracy_score(y_val, pred_log)

acuracia['Familiares'] = [acuracia_tree, acuracia_knn, acuracia_rl]

acuracia

,Modelos,Somente Numéricos,Sex e OHE,Escala,Familiares
0,Árvore de decisão,0.616949,0.738983,0.738983,0.749153
1,KNN,0.667797,0.705085,0.800000,0.789831
2,Regressão logística,0.725424,0.816949,0.816949,0.810169


Apenas a árvore de decisão conseguiu aumentar a acurácia, ainda que</br>
o valor alcançado nçao apresentou varação significativa.</br>
Como os outros valores estão inferiores e o citado está bem abaixo, os</br>
resultados não serão exportados para o Kaggle.

## 11) SELECIONANDO AS MELHORES VARIÁVEIS PARA O PROJETO
---

In [151]:
train.corr()

,Survived,Pclass,Sex,Age,Fare,Embarked_S,Embarked_C,Embarked_Q,Sozinho,Familiares
Survived,1.000000,-0.338481,-0.543351,-0.069809,0.257307,-0.149683,0.168240,0.003650,-0.203367,0.016639
Pclass,-0.338481,1.000000,0.131900,-0.331339,-0.549500,0.074053,-0.243292,0.221009,0.135207,0.065997
Sex,-0.543351,0.131900,1.000000,0.084153,-0.182333,0.119224,-0.082853,-0.074115,0.303646,-0.200988
Age,-0.069809,-0.331339,0.084153,1.000000,0.091566,-0.019336,0.032024,-0.013855,0.179775,-0.248512
Fare,0.257307,-0.549500,-0.182333,0.091566,1.000000,-0.162184,0.269335,-0.117216,-0.271832,0.217138
Embarked_S,-0.149683,0.074053,0.119224,-0.019336,-0.162184,1.000000,-0.782742,-0.499421,0.029074,0.077359
Embarked_C,0.168240,-0.243292,-0.082853,0.032024,0.269335,-0.782742,1.000000,-0.148258,-0.095298,-0.046215
Embarked_Q,0.003650,0.221009,-0.074115,-0.013855,-0.117216,-0.499421,-0.148258,1.000000,0.086464,-0.058592
Sozinho,-0.203367,0.135207,0.303646,0.179775,-0.271832,0.029074,-0.095298,0.086464,1.000000,-0.690922
Familiares,0.016639,0.065997,-0.200988,-0.248512,0.217138,0.077359,-0.046215,-0.058592,-0.690922,1.000000


In [152]:
fig, ax = plt.subplots(figsize=(10,6))
sns.heatmap(train.corr(), annot=True, ax=ax)
plt.show()

A maior relação entre as variáveis está se dando entre Fare e Pclass.
### Relação entre Fare e Pclass

In [153]:
fig, ax = plt.subplots(figsize=(10,6))

sns.boxplot(data=train, x='Pclass', y='Fare', hue='Survived')
ax.set_title('Pclass x Fare')

plt.show()

Entre os passageiros da prieira classe o preço pago na passagem influenciou</br>
muito a taxa de sobrevivência.
### Examinando a taxa de sobrevivência em cada classe
Porcentagem de sobreviventes em cada classe.

In [154]:
train.groupby('Pclass')['Survived'].mean()

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

Valor médio dos passageiros que sobreviveram e que não sobreviveram para cada classe.</br>
A escala dos dados já foi modificada.

In [155]:
train.groupby(['Pclass', 'Survived'])['Fare'].mean()

Pclass  Survived
1       0           2.175430
        1           3.514735
2       0           0.214734
        1           0.329217
3       0          -0.033991
        1          -0.032885
Name: Fare, dtype: float64

Há uma grande diferença do valor na primeira classe, bem menos na segunda e inconclusivo na terceira.

In [156]:
train.groupby(['Pclass', 'Survived'])['Fare'].agg(['min', 'mean', 'max', 'sum', 'count'])

min      mean        max         sum  count
Pclass Survived                                                  
1      0        -0.626005  2.175430  10.764405  174.034396     80
       1         0.496977  3.514735  21.562738  478.003980    136
2      0        -0.626005  0.214734   2.557247   20.829222     97
       1        -0.171255  0.329217   2.189115   28.641921     87
3      0        -0.626005 -0.033991   2.386174  -12.644606    372
       1        -0.626005 -0.032885   1.820802   -3.913372    119

### Considerando uma cardinalidade para o porto de embarque e eliminando o One-Hot-Encoding
Importação novamente da variável Embarked e eliminação do One-Hote-Encoding.</br>
Os portos apresentam variação no tempo de viagem e isto será levado em consideração</br>
para a criação de uma cardinalidade entre os dados.</br>
Serão colocados 'S' como 0, 'C' como 1 e 'Q' como 2.</br>
Os valores nulos são substituídos pela moda.

In [157]:
train2 = pd.read_csv('DATA/train.csv')
test2 = pd.read_csv('DATA/test.csv')

train2['Embarked'] = train2['Embarked'].fillna('S')

train2['Embarked'] = train2['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2}).astype(int)
test2['Embarked'] = test2['Embarked'].replace({'S': 0, 'C': 1, 'Q': 2}).astype(int)

train = train.assign(Embarked=train2['Embarked'].values)
test = test.assign(Embarked=test2['Embarked'].values)

train.drop(columns = ['Embarked_C', 'Embarked_Q', 'Embarked_S'], inplace=True)
test.drop(columns = ['Embarked_C', 'Embarked_Q', 'Embarked_S'], inplace=True)

In [158]:
train.head(2)

,Survived,Pclass,Sex,Age,Fare,Sozinho,Familiares,Embarked
0,0,3,1,-0.592240,-0.312011,0,1,0
1,1,1,0,0.638529,2.461242,0,1,1


## 12) APLICAÇÃO DOS MODELOS
---

In [159]:
X = train.drop('Survived', axis=1)
y = train['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

clf_tree = tree.DecisionTreeClassifier(random_state = 42)
clf_tree = clf_tree.fit(X_train, y_train)
pred_tree = clf_tree.predict(X_val)

clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_knn .fit(X_train, y_train)
pred_knn = clf_knn .predict(X_val)

clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)
pred_log = clf.predict(X_val)

acuracia_tree = accuracy_score(y_val, pred_tree)
acuracia_knn = accuracy_score(y_val, pred_knn)
acuracia_rl = accuracy_score(y_val, pred_log)

acuracia['Porto'] = [acuracia_tree, acuracia_knn, acuracia_rl]

acuracia

,Modelos,Somente Numéricos,Sex e OHE,Escala,Familiares,Porto
0,Árvore de decisão,0.616949,0.738983,0.738983,0.749153,0.766102
1,KNN,0.667797,0.705085,0.800000,0.789831,0.816949
2,Regressão logística,0.725424,0.816949,0.816949,0.810169,0.810169


Neste caso não hove aumento da acurácia para Regressão logística.</br>
A Árvore de decisão e KNN apresentaram aumento e para este exemplo houve </br>
auteração no número de vizinhos no KNN, anteriormente eram 3 e agora foram 5.</br>
Como resultado, a acurária utrapaçou o valor obtido pela Árvore de decisão.
### Exportação para o Kaggle
Novamente o KNN foi escolhido para apresentar o resultados nos dados de teste.

In [160]:
df_desafio = pd.DataFrame()
df_desafio['PassengerId'] = test['PassengerId']
X_test = test.drop(['PassengerId'], axis = 1)
y_pred = clf_knn.predict(X_test)
df_desafio['Survived'] = y_pred

df_desafio.to_csv('Export/porto.csv', index = False)

<img src="IMG/im4.png" width="500" height="800">

Houve um pequeno aumento para o KNN mas ainda não atingiu a maior validação.
## 13) APLICAÇÃO DE NOVOS MODELOS AOS DADOS
---
Serão adicionados o Random Forest e o MLP.

In [161]:
aux = pd.DataFrame({
    'Modelos':['Random Forest', 'MLP'],
    'Somente Numéricos': [0,0],
    'Sex e OHE':[0,0],
    'Escala':[0,0],
    'Familiares':[0,0],
    'Porto':[0,0],
})

acuracia = pd.concat([acuracia, aux], axis=0)

In [163]:
X = train.drop(['Survived'], axis = 1)
y = train.Survived

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

clf_rf = RandomForestClassifier(random_state=42)
clf_rf = clf_rf.fit(X_train, y_train)
pred_rf = clf_rf.predict(X_val)

clf_mlp = MLPClassifier(random_state=42, max_iter=5000)
clf_mlp = clf_mlp.fit(X_train, y_train)
pred_mlp = clf_mlp.predict(X_val)

acuracia_rf = accuracy_score(y_val, pred_rf)
acuracia_mlp = accuracy_score(y_val, pred_mlp)

acuracia['Algs'] = [acuracia_tree, acuracia_knn, acuracia_rl, acuracia_rf, acuracia_mlp]

acuracia

,Modelos,Somente Numéricos,Sex e OHE,Escala,Familiares,Porto,Algs
0,Árvore de decisão,0.616949,0.738983,0.738983,0.749153,0.766102,0.766102
1,KNN,0.667797,0.705085,0.800000,0.789831,0.816949,0.816949
2,Regressão logística,0.725424,0.816949,0.816949,0.810169,0.810169,0.810169
0,Random Forest,0.000000,0.000000,0.000000,0.000000,0.000000,0.786441
1,MLP,0.000000,0.000000,0.000000,0.000000,0.000000,0.827119


### Exportação ao Kaggle
O MLP apresentou resultados de acurácia ainda maior então será utilizado no envio de resultados ao Kaggle.

In [164]:
df_desafio = pd.DataFrame()
df_desafio['PassengerId'] = test['PassengerId']
X_test = test.drop(['PassengerId'], axis = 1)
y_pred = clf_mlp.predict(X_test)
df_desafio['Survived'] = y_pred

df_desafio.to_csv('Export/algoritmos.csv', index = False)

<img src="IMG/im5.png" width="500" height="800">

Apesar da acurácia nos dados de teste ser o maior valor já obtido</br>
o resultado nos dados de treino foram bem mais baixos e isso pode se dar </br>
devido a um Overfitting.

## 14) TUNING DOS PARÂMETROS
---
Os parâmetros de cada modelo serão variados e selecionados de acordo com a maior acurácia.<br>
Para isso o tamanho da divisão dos dados para teste e validação deve ser menor.

In [165]:
X = train.drop('Survived', axis=1)
y = train.Survived

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Grupos de possíveis parâmetros para o teste do resultado
Regressão logística

In [166]:
clf_rl = LogisticRegression(random_state=42)

parametros_rl = {
    'penalty': ['l1','l2'],
    'C': [0.05,0.1,0.15,0.2],
    'solver': ['lbfgs','liblinear','saga'],
    'max_iter': [100,1000,5000,10000, 50000]
}

Random Forest

In [167]:
clf_rf = RandomForestClassifier(random_state=42)

parametros_rf = {
    'n_estimators': [30, 50, 60, 70, 80, 100, 150],
    'criterion': ['gini','entropy','log_loss'],
    'max_depth': [2,4,6,8,None],
    'max_features': ['sqrt','log2',None]
}

MLP

In [168]:
clf_mlp = MLPClassifier(random_state=42)

parametros_mlp = {
    'solver':  ['lbfgs','sgd','adam'],
    'alpha': [10.0**(-1),10.0**(-5),10.0**(-7),10.0**(-10)],
    'max_iter': [200,500,1000,5000]
}

#### Função para acompanhamento do tempo de execução

In [169]:
def mostra_hora():
    agora = datetime.now()
    print(str(agora.hour)+':'+str(agora.minute)+':'+str(agora.second))

### Obtendo os melhores parâmetro
Acompanhamento dos tempos para a variação dos parâmetros em cada algoritmo.

In [170]:
#Regressão Logística
mostra_hora()
kfold_rl = KFold(shuffle=True, random_state=42,n_splits=8)
grid_search_rl = GridSearchCV(estimator=clf_rl,param_grid=parametros_rl, scoring='accuracy',cv=kfold_rl)
grid_search_rl = grid_search_rl.fit(X_train,y_train)
mostra_hora()

22:28:20
22:28:26


In [171]:
# RandomForest
mostra_hora()
kfold_rf = KFold(shuffle=True, random_state=42,n_splits=8)
grid_search_rf = GridSearchCV(estimator=clf_rf,param_grid=parametros_rf,scoring='accuracy',cv=kfold_rf)
grid_search_rf = grid_search_rf.fit(X_train,y_train)
mostra_hora()

22:28:26
22:32:4


In [172]:
# MLPClassifier
mostra_hora()
kfold_mlp = KFold(shuffle=True, random_state=42,n_splits=8)
grid_search_mlp = GridSearchCV(estimator=clf_mlp,param_grid=parametros_mlp,scoring='accuracy',cv=kfold_mlp)
grid_search_mlp = grid_search_mlp.fit(X_train,y_train)
mostra_hora()

22:32:4
22:36:32


### Acurácia

In [173]:
reg_log = grid_search_rl.best_score_
rand_forest = grid_search_rf.best_score_
mlp = grid_search_mlp.best_score_

acuracia = pd.DataFrame({
    'Modelos':['Regressão Logística', 'Random Forest', 'MLP'],
    'Acurácia':[reg_log,rand_forest,mlp]
})

acuracia

,Modelos,Acurácia
0,Regressão Logística,0.806180
1,Random Forest,0.834270
2,MLP,0.817416


### Melhores parâmetros

In [174]:
# Regressão logística
grid_search_rl.best_params_

{'C': 0.1, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'saga'}

In [175]:
# RandomForest
grid_search_rf.best_params_

{'criterion': 'gini', 'max_depth': 6, 'max_features': None, 'n_estimators': 30}

In [176]:
# MLPClassifier
grid_search_mlp.best_params_

{'alpha': 0.1, 'max_iter': 500, 'solver': 'adam'}

### Exportação Kaggle
O Random Forest com os parâmetros:</br>
{'criterion': 'gini', 'max_depth': 6, 'max_features': None, 'n_estimators': 30}</br>
Apresentou o maior valor para a acurácia.</br>
Logo, será escolhido para enviar a resolução ao Kaggle.

In [177]:
rf_model = RandomForestClassifier(
                                    criterion='gini',
                                    max_depth=6,
                                    max_features=None,
                                    n_estimators=30,
                                    random_state=42
                                )
rf_model = rf_model.fit(X_train, y_train)

In [179]:
df_desafio = pd.DataFrame()
df_desafio['PassengerId'] = test['PassengerId']
X_test = test.drop(['PassengerId'], axis = 1)
y_pred = rf_model.predict(X_test)
df_desafio['Survived'] = y_pred

df_desafio.to_csv('Export/tuning.csv', index = False)

<img src="IMG/im6.png" width="500" height="800">

Random forest com melhores parâmetros apresenta o maior resultado obtido.